In [1]:
import pandas as pd

# read the data

ori_data = pd.read_csv('X23241.csv')
ori_data.drop(ori_data.columns.values[0], axis=1, inplace=True)

X_train = pd.read_csv('X23241.csv').values[:,1:]
y_train = pd.read_csv('Y23241.csv').values[:,1].ravel().astype(int)

In [2]:
from sklearn.model_selection import train_test_split

# split the data
X_train_sub, X_valid, y_train_sub, y_valid = \
    train_test_split(X_train, y_train, test_size=0.2, random_state=1, stratify=y_train)

print('Train/Valid sizes:', y_train_sub.shape[0], y_valid.shape[0])

Train/Valid sizes: 18592 4649


In [4]:
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing

# pay attention to the data leaking, we can not use information on validation/test set
scaler = preprocessing.StandardScaler().fit(X_train_sub[:,0:9]) # only use pca for numerical variables

X_scaled_sub = scaler.transform(X_train_sub[:,0:9]) 
X_scaled_valid = scaler.transform(X_valid[:,0:9]) #only use the information from training set

pca = PCA(n_components=5)
pca.fit(X_scaled_sub)
X_new_train = pca.transform(X_scaled_sub)
X_new_valid = pca.transform(X_scaled_valid)

X_std_train = np.hstack((X_new_train, X_train_sub[:,10:]))
X_std_valid = np.hstack((X_new_valid, X_valid[:,10:]))

X_std_train = pd.DataFrame(data = X_std_train)
X_std_valid = pd.DataFrame(data = X_std_valid)
y_train_sub = pd.DataFrame(data = y_train_sub)
y_valid = pd.DataFrame(data = y_valid)


X_std_train.to_csv("pcaX_train.csv", index = False)
X_std_valid.to_csv("pcaX_test.csv", index = False)
y_train_sub.to_csv("pcay_train.csv", index = False)
y_valid.to_csv("pcay_test.csv", index = False)

print(pca.explained_variance_ratio_) # show explained variance ratio

print(pca.components_) # show principal components

[0.28826454 0.22252553 0.19245387 0.10603844 0.06588928]
[[-0.11447395 -0.48802539  0.10629545 -0.10310452 -0.07347723 -0.13478228
   0.44601255 -0.3895188  -0.59378482]
 [-0.34697426  0.16817915  0.57992905  0.02601582  0.27997311 -0.34000309
  -0.40117772 -0.39809669  0.03030707]
 [ 0.39959507 -0.09901385  0.05694173  0.6562734   0.58998156  0.16276772
   0.07404582 -0.07113666 -0.10708937]
 [ 0.33901863  0.03472771  0.32713694 -0.25191764 -0.19657656  0.70809979
  -0.21400659 -0.35245072 -0.05754382]
 [-0.73272082 -0.22959296 -0.131311    0.08176409  0.24943204  0.55950308
  -0.01647461  0.07538013  0.07256517]]
